In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
import gradio as gr

In [10]:
# Step 1: Sample Poem Text (you can load from .txt too)
with open("/content/drive/MyDrive/Divya/poem.txt", "r") as file:
    poem = file.read().lower()
import re
poem = re.sub(r'[^a-z \n]', '', poem.lower())

# Step 2: Character Tokenization
chars = sorted(set(poem))
char_to_index = {c: i for i, c in enumerate(chars)}
index_to_char = {i: c for c, i in char_to_index.items()}
vocab_size = len(chars)

# Create input-output pairs
sequence_length = 10
sequences = []
targets = []

for i in range(len(poem) - sequence_length):
    seq = poem[i:i+sequence_length]
    target = poem[i+sequence_length]
    sequences.append([char_to_index[c] for c in seq])
    targets.append(char_to_index[target])

x = np.array(sequences)
y = to_categorical(targets, num_classes=vocab_size)
x = x.reshape((x.shape[0], x.shape[1], 1))  # shape: (samples, time_steps, features)

# Step 3: Build LSTM Model
model = Sequential()
model.add(LSTM(128, activation='relu', input_shape=(sequence_length, 1)))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.fit(x, y, epochs=300, verbose=0)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [11]:
import pickle

with open('/content/drive/MyDrive/Divya/poemgen.pkl', 'wb') as file:
    pickle.dump(model,file)

In [12]:
with open('/content/drive/MyDrive/Divya/poemgen.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

In [13]:
# Step 4: Text Generation Function
def generate_poem(seed, num_chars):
    result = seed
    for _ in range(num_chars):
        input_seq = [char_to_index.get(c, 0) for c in seed[-sequence_length:]]
        input_seq = pad_sequences([input_seq], maxlen=sequence_length)
        input_seq = np.reshape(input_seq, (1, sequence_length, 1))
        predicted_probs = model.predict(input_seq, verbose=0)
        predicted_index = np.argmax(predicted_probs)
        predicted_char = index_to_char[predicted_index]
        result += predicted_char
        seed += predicted_char
    return result

# Step 5: Gradio App
demo = gr.Interface(
    fn=generate_poem,
    inputs=[
        gr.Textbox(label="Seed Text", value="roses are "),
        gr.Slider(10, 100, value=50, step=1, label="Number of Characters to Generate")
    ],
    outputs="text",
    title="Poem Generator using LSTM (Character-Level)",
    description="Enter a starting phrase and let the model generate poetic text character by character."
)

demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://54b7eb73edcdd8ee18.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
